In [2]:
#include <aio.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <errno.h>
#include <string.h>
#include <signal.h>

In [ ]:

aio_read(&cb)
#需要定义以下结构体
struct aiocb cb;
// gcc -o test aio_signal.c -lrt 

In [5]:
    struct aiocb cb;
    char sbuf[100];
    #填充aiocb结构体

    cb.aio_fildes = 0;
	cb.aio_buf = sbuf;
	cb.aio_nbytes = 100;
	cb.aio_offset = 0;

	cb.aio_sigevent.sigev_value.sival_ptr = &cb;
	cb.aio_sigevent.sigev_notify = SIGEV_SIGNAL;
	cb.aio_sigevent.sigev_signo = SIGUSR1;
    aio_read(&cb);

In [ ]:
#   aio_read(&cb) 异步执行 也就是非阻塞，  怎么读取sbuf的数据？
	struct aiocb cb;
	char sbuf[100];
	int ret;

	bzero(&cb, sizeof(cb));

	cb.aio_fildes = 0;
	cb.aio_buf = sbuf;
	cb.aio_nbytes = 100;
	cb.aio_offset = 0;

	cb.aio_sigevent.sigev_value.sival_ptr = &cb;
	cb.aio_sigevent.sigev_notify = SIGEV_SIGNAL;
	cb.aio_sigevent.sigev_signo = SIGUSR1;
	ret = aio_read(&cb);
# 后面主程序继续执行任务，  当数据完成进入用户空间时，SIGUSR1信号发出，
#程序处理SIGUSR1信号  
	if (ret == -1) {
		perror("aio_read");
		exit(1);
	}
	int i = 0;
	while (1) {
		printf("%d\n",i++);
		sleep(3);
	}



In [ ]:
#程序处理SIGUSR1信号  
    void async_read(int s, siginfo_t * info, void * context)
    {
        #read aio_buf中的数据
    }

    struct sigaction act;
	sigemptyset(&act.sa_mask);
	act.sa_flags = SA_RESTART | SA_SIGINFO;
	act.sa_sigaction = async_read;
	sigaction(SIGUSR1, &act, NULL);
   


In [ ]:
#在async_read(int s, siginfo_t * info, void * context)函数中怎么读取 aio_buf中的数据？
    void async_read(int s, siginfo_t * info, void * context)
    {
        struct aiocb *ptr = 
        (struct aiocb *)info->si_value.sival_ptr;
        printf("read=%s", (char *)ptr->aio_buf);    
    }
si_value联系struct sigaction与struct aiocb的桥梁

In [ ]:
#完整代码
void async_read(int s, siginfo_t * info, void * context)
{
	sigval_t
	struct aiocb *ptr = 
		(struct aiocb *)info->si_value.sival_ptr;
	printf("read=%s", (char *)ptr->aio_buf);    
}

int main(void)
{
	struct aiocb cb;
	char sbuf[100];
	int ret;
	struct sigaction act;
	sigemptyset(&act.sa_mask);
	act.sa_flags = SA_RESTART | SA_SIGINFO;  //指定了带三个参数信号处理函数
	act.sa_sigaction = async_read;  //处理函数的地址

	sigaction(SIGUSR1, &act, NULL); //注册信号处理函数

	bzero(&cb, sizeof(cb));

	cb.aio_fildes = 0;
	cb.aio_buf = sbuf;
	cb.aio_nbytes = 100;
	cb.aio_offset = 0;

	cb.aio_sigevent.sigev_value.sival_ptr = &cb; //送个信号处理函数的那个结构体
	cb.aio_sigevent.sigev_notify = SIGEV_SIGNAL;  //异步通知为发信号
	cb.aio_sigevent.sigev_signo = SIGUSR1;  // 发出的信号
	ret = aio_read(&cb);
	if (ret == -1) {
		perror("aio_read");
		exit(1);
	}
	int i = 0;
	while (1) {
		printf("%d\n",i++);
		sleep(3);
	}

	return 0;
}

// gcc -o test aio_signal.c -lrt 